pip install opencv-python
pip install imutils
pip install dlib

In [3]:
from imutils.face_utils import FaceAligner, rect_to_bb
import imutils
import dlib
import cv2
import matplotlib.pyplot as plt
from glob import glob
import os
import requests

from PIL import Image, ImageOps
from pillow_heif import register_heif_opener

register_heif_opener()

In [4]:
# capture issues to resolve in the future:
# buttons as faces
# other people
capture_issues = ['20210614_183300.jpg', '20210829_093342.jpg',
                  '20220301_222916.jpg','20220626_135456.jpg', 
                  '20220917_163316.jpg', '20220925_190928.jpg',
                  '20221128_194535.jpg', 'IMG_5303.HEIC',
                  'IMG_5011.HEIC', 'IMG_4758.HEIC',
                  'IMG_4532.HEIC']



need_to_be_renamed = [
               '327766_2110888256203_2015606625_o.jpg',
               '13483308_10206732714060887_9206378074821374390_o.jpg',
               '13938075_10207037654724213_7700935497558972159_o.jpg',
                'IMG_2267.JPG',
                'AirBrush_20210330203813.jpg']

image_issues = capture_issues + need_to_be_renamed

In [5]:
from imutils.face_utils.helpers import FACIAL_LANDMARKS_68_IDXS
from imutils.face_utils.helpers import FACIAL_LANDMARKS_5_IDXS
from imutils.face_utils.helpers import shape_to_np
import numpy as np
import cv2

class FaceAligner:
	def __init__(self, predictor, desiredLeftEye=(0.35, 0.35),
		desiredFaceWidth=256, desiredFaceHeight=None):
		# store the facial landmark predictor, desired output left
		# eye position, and desired output face width + height
		self.predictor = predictor
		self.desiredLeftEye = desiredLeftEye
		self.desiredFaceWidth = desiredFaceWidth
		self.desiredFaceHeight = desiredFaceHeight

		# if the desired face height is None, set it to be the
		# desired face width (normal behavior)
		if self.desiredFaceHeight is None:
			self.desiredFaceHeight = self.desiredFaceWidth

	def align(self, image, gray, rect):
		# convert the landmark (x, y)-coordinates to a NumPy array
		shape = self.predictor(gray, rect)
		shape = shape_to_np(shape)
		
		#simple hack ;)
		if (len(shape)==68):
			# extract the left and right eye (x, y)-coordinates
			(lStart, lEnd) = FACIAL_LANDMARKS_68_IDXS["left_eye"]
			(rStart, rEnd) = FACIAL_LANDMARKS_68_IDXS["right_eye"]
		else:
			(lStart, lEnd) = FACIAL_LANDMARKS_5_IDXS["left_eye"]
			(rStart, rEnd) = FACIAL_LANDMARKS_5_IDXS["right_eye"]
			
		leftEyePts = shape[lStart:lEnd]
		rightEyePts = shape[rStart:rEnd]

		# compute the center of mass for each eye
		leftEyeCenter = leftEyePts.mean(axis=0).astype("int")
		rightEyeCenter = rightEyePts.mean(axis=0).astype("int")

		# compute the angle between the eye centroids
		dY = rightEyeCenter[1] - leftEyeCenter[1]
		dX = rightEyeCenter[0] - leftEyeCenter[0]
		angle = np.degrees(np.arctan2(dY, dX)) - 180

		# compute the desired right eye x-coordinate based on the
		# desired x-coordinate of the left eye
		desiredRightEyeX = 1.0 - self.desiredLeftEye[0]

		# determine the scale of the new resulting image by taking
		# the ratio of the distance between eyes in the *current*
		# image to the ratio of distance between eyes in the
		# *desired* image
		dist = np.sqrt((dX ** 2) + (dY ** 2))
		desiredDist = (desiredRightEyeX - self.desiredLeftEye[0])
		desiredDist *= self.desiredFaceWidth
		scale = desiredDist / dist

		# compute center (x, y)-coordinates (i.e., the median point)
		# between the two eyes in the input image
		eyesCenter = (int((leftEyeCenter[0] + rightEyeCenter[0]) // 2),
            int((leftEyeCenter[1] + rightEyeCenter[1]) // 2))

		# grab the rotation matrix for rotating and scaling the face
		M = cv2.getRotationMatrix2D(eyesCenter, angle, scale)

		# update the translation component of the matrix
		tX = self.desiredFaceWidth * 0.5
		tY = self.desiredFaceHeight * self.desiredLeftEye[1]
		M[0, 2] += (tX - eyesCenter[0])
		M[1, 2] += (tY - eyesCenter[1])

		# apply the affine transformation
		(w, h) = (self.desiredFaceWidth, self.desiredFaceHeight)
		output = cv2.warpAffine(image, M, (w, h),
			flags=cv2.INTER_CUBIC)

		# return the aligned face
		return output

In [6]:
landmark_name = "shape_predictor_68_face_landmarks.dat"
if not os.path.isfile(landmark_name):
    landmarks_url = "https://github.com/italojs/facial-landmarks-recognition/raw/master/shape_predictor_68_face_landmarks.dat"
    out = requests.get(landmarks_url)
    open(landmark_name, "wb").write(out.content)


In [7]:
# get paths
paths = glob('original_faces/*')
# remove MP$
paths = [path for path in paths if not path.endswith('MP4')]
failed_paths = []
verbose = False
save = True

size = 1024

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
fa = FaceAligner(predictor, desiredFaceWidth=size, desiredLeftEye=(.42,.5))

def to_aligned_path(image_path):
    aligned_base = f'aligned_faces/{os.path.basename(image_path)}'
    aligned_path =  os.path.splitext(aligned_base)[0] + '.jpg'
    return aligned_path

for image_path in paths:
    aligned_path = to_aligned_path(image_path)
    if os.path.isfile(aligned_path) or  os.path.basename(image_path) in image_issues:
        continue
    try:
        # load the input image, resize it, and convert it to grayscale
        # load and transform image
        image = Image.open(image_path).convert('RGB')
        image = ImageOps.exif_transpose(image)
        image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
        image = imutils.resize(image, width=800)
        print(f'successful load of {image_path}')
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        # show the original input image and detect faces in the grayscale
        # image
        cv2.imshow("Input", image)
        rects = detector(gray, 2)



        # loop over the face detections
        rect = rects[0]
        # extract the ROI of the *original* face, then align the face
        # using facial landmarks
        (x, y, w, h) = rect_to_bb(rect)
        faceOrig = imutils.resize(image[y:y + h, x:x + w], width=size)
        faceAligned = fa.align(image, gray, rect)
        if save:
            cv2.imwrite(aligned_path, faceAligned)
        # display the output images
        if verbose:
            plt.subplot(1,2,1)
            plt.imshow(faceOrig)
            plt.subplot(1,2,2)
            plt.imshow(faceAligned)
            plt.show()
    except Exception as e:
        print(e)
        failed_paths.append(image_path)


successful load of original_faces/IMG_5750.HEIC
successful load of original_faces/IMG_4907.HEIC
successful load of original_faces/IMG_4287.HEIC
successful load of original_faces/IMG_6255.HEIC
successful load of original_faces/IMG_5029.HEIC
successful load of original_faces/20210522_144933.jpg

successful load of original_faces/IMG_4100.HEIC
successful load of original_faces/IMG_4803.HEIC
successful load of original_faces/IMG_3915.HEIC
successful load of original_faces/IMG_6957.HEIC
successful load of original_faces/IMG_3893.HEIC

successful load of original_faces/IMG_6845.HEIC
successful load of original_faces/IMG_5910.HEIC
cannot identify image file 'original_faces/IMG_4523.MP4'
successful load of original_faces/IMG_4182.HEIC
successful load of original_faces/IMG_6279.HEIC
successful load of original_faces/IMG_5863.HEIC
successful load of original_faces/20201031_184422.jpg

successful load of original_faces/IMG_6961.HEIC
successful load of original_faces/IMG_5571.HEIC
successful load 

In [11]:
def get_date_taken(path):
    exif = Image.open(path).getexif()
    if not exif:
        print('Image {0} does not have EXIF data.'.format(path))
        return None
    return exif.get(306)

dates = []
for path in paths:
    if os.path.basename(path) in image_issues:
        continue
    date = get_date_taken(path)
    if date is not None:
        dates.append((date, to_aligned_path(path)))
# sort the dates
dates = sorted(dates, key = lambda x: x[0])

Image original_faces/cfdc965cee-result-20210822214218.jpeg does not have EXIF data.


In [22]:
image_paths = [i[1] for i in dates]
frame = cv2.imread(image_paths[0])
height, width, layers = frame.shape


In [13]:
output_filename = 'aligned_faces_video.mp4'
fps = 9
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
video = cv2.VideoWriter(output_filename, fourcc, fps, (width,height))
for path in image_paths:
    video.write(cv2.imread(path))

cv2.destroyAllWindows()
video.release()

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
[ WARN:0@1209.063] global loadsave.cpp:248 findDecoder imread_('aligned_faces/DSC_0492.jpg'): can't open/read file: check file path/integrity
[ WARN:0@1209.595] global loadsave.cpp:248 findDecoder imread_('aligned_faces/20201031_184422.jpg'): can't open/read file: check file path/integrity
[ WARN:0@1209.932] global loadsave.cpp:248 findDecoder imread_('aligned_faces/20210111_152931.jpg'): can't open/read file: check file path/integrity
[ WARN:0@1209.947] global loadsave.cpp:248 findDecoder imread_('aligned_faces/20210116_142229.jpg'): can't open/read file: check file path/integrity
[ WARN:0@1210.032] global loadsave.cpp:248 findDecoder imread_('aligned_faces/20210206_103216.jpg'): can't open/read file: check file path/integrity
[ WARN:0@1210.150] global loadsave.cpp:248 findDecoder imread_('aligned_faces/20210306_113702.jp

In [52]:
# Create average photo
def create_average_image(image_paths):        
    average_image = None
    total_images = 0
    # Iterate over the rest of the images
    for path in image_paths:
        # Read the image
        image = cv2.imread(path)
        try:
            image = image.astype(np.float32)
            total_images += 1
        except:
            continue
        # Add the image to the average
        if average_image is None:
            average_image = image
        else:
            average_image += image
    if average_image is not None:
        # Divide by the number of images to get the average
        average_image /= total_images

        # Convert back to 8-bit
        average_image = average_image.astype(np.uint8)
        return average_image

average_image = create_average_image(image_paths)
# Save the averaged image
cv2.imwrite('average_images/overall_average_image.jpg', average_image)

[ WARN:0@2423.395] global loadsave.cpp:248 findDecoder imread_('aligned_faces/DSC_0492.jpg'): can't open/read file: check file path/integrity
[ WARN:0@2423.593] global loadsave.cpp:248 findDecoder imread_('aligned_faces/20201031_184422.jpg'): can't open/read file: check file path/integrity
[ WARN:0@2423.712] global loadsave.cpp:248 findDecoder imread_('aligned_faces/20210111_152931.jpg'): can't open/read file: check file path/integrity
[ WARN:0@2423.717] global loadsave.cpp:248 findDecoder imread_('aligned_faces/20210116_142229.jpg'): can't open/read file: check file path/integrity
[ WARN:0@2423.745] global loadsave.cpp:248 findDecoder imread_('aligned_faces/20210206_103216.jpg'): can't open/read file: check file path/integrity
[ WARN:0@2423.783] global loadsave.cpp:248 findDecoder imread_('aligned_faces/20210306_113702.jpg'): can't open/read file: check file path/integrity
[ WARN:0@2423.825] global loadsave.cpp:248 findDecoder imread_('aligned_faces/20210322_174406.jpg'): can't open/r

True

In [53]:
from datetime import datetime
from collections import defaultdict
images_by_month = defaultdict(list)
images_by_quarter = defaultdict(list)
# Iterate over the dates and images
for date_str, image_path in dates:
    # Parse the date
    date = datetime.strptime(date_str, '%Y:%m:%d %H:%M:%S')

    # Calculate the quarter
    quarter = (date.month - 1) // 3 + 1

    month_key = (date.year, date.month)
    quarter_key = (date.year, f"Q{quarter}")
    
    # Add the image to the appropriate list
    images_by_month[month_key].append(image_path)
    images_by_quarter[quarter_key].append(image_path)

    
for key, images in images_by_quarter.items():
    # Create the output filename
    output_filename = 'average_images/{0}_{1}_average_image_{2}images.jpg'.format(key[0], key[1], len(images))
    
    # Create the average image for the month
    average_image = create_average_image(images)
    if average_image is not None:
        # Save the image
        cv2.imwrite(output_filename, average_image)


[ WARN:0@2581.372] global loadsave.cpp:248 findDecoder imread_('aligned_faces/DSC_0492.jpg'): can't open/read file: check file path/integrity
[ WARN:0@2581.668] global loadsave.cpp:248 findDecoder imread_('aligned_faces/20201031_184422.jpg'): can't open/read file: check file path/integrity
[ WARN:0@2581.805] global loadsave.cpp:248 findDecoder imread_('aligned_faces/20210111_152931.jpg'): can't open/read file: check file path/integrity
[ WARN:0@2581.810] global loadsave.cpp:248 findDecoder imread_('aligned_faces/20210116_142229.jpg'): can't open/read file: check file path/integrity
[ WARN:0@2581.843] global loadsave.cpp:248 findDecoder imread_('aligned_faces/20210206_103216.jpg'): can't open/read file: check file path/integrity
[ WARN:0@2581.887] global loadsave.cpp:248 findDecoder imread_('aligned_faces/20210306_113702.jpg'): can't open/read file: check file path/integrity
[ WARN:0@2581.931] global loadsave.cpp:248 findDecoder imread_('aligned_faces/20210322_174406.jpg'): can't open/r

## Future Explorations
URL: https://archive.is/lQXdx